In [23]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from xgboost import XGBClassifier

# 1. Version1 (half NC2)

In [24]:
data_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver1(half NC)/features.csv'
data = pd.read_csv(data_dir)

In [25]:
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_C4,spectral_entropy_T8,spectral_entropy_P7,spectral_entropy_P3,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,label
0,-0.418358,0.332468,-0.342700,-0.434762,-0.363083,1.541473,3.803859,-0.366574,-0.361034,-0.317023,...,2.769378,3.243881,3.314180,2.991150,2.867881,2.897658,3.226472,3.215344,3.264944,2
1,-0.553936,0.645437,-0.499525,-0.413223,-0.233330,1.566611,3.691952,-0.446604,-0.195897,-0.095939,...,3.014827,3.851837,3.643615,3.271922,3.141803,3.162673,3.711791,3.533822,3.642025,2
2,1.028320,-0.381386,0.390925,0.612009,0.007393,-1.033033,-0.263930,-0.269617,-0.003700,-1.806611,...,3.997216,3.996011,4.050526,4.029060,4.035325,4.044881,4.029477,4.004199,4.097106,2
3,1.024712,-0.350840,0.469474,0.555023,-0.059696,-0.864880,0.112843,-0.260772,-0.060767,-1.878602,...,3.781055,3.795044,3.863689,3.835626,3.824710,3.850665,3.836143,3.776048,3.924873,2
4,-0.007232,0.071214,-0.244308,-0.243306,-0.211503,-0.158064,4.221788,-0.248971,-0.280709,-0.297559,...,2.857713,3.287834,3.464297,3.088678,2.997639,3.016220,3.331265,3.395650,3.406698,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5107,-0.460583,-0.610322,-0.259787,-0.600999,-0.280894,-0.350454,-0.400338,0.169987,0.024974,0.239657,...,5.559681,5.142015,4.995880,5.014595,4.944117,5.109019,5.099218,4.983094,4.952925,0
5108,-0.976086,-1.028429,0.414336,-0.563494,-0.408615,-0.306153,0.013982,0.673318,-0.217690,0.330940,...,5.608489,5.337052,5.162284,5.187759,5.117956,5.293531,5.287068,5.148920,5.126615,0
5109,0.519969,-0.047965,-0.707030,-0.717179,-1.108096,-1.164509,-1.042517,1.417827,-0.897542,0.410701,...,5.676561,5.439604,5.378844,5.437639,5.277062,5.301990,5.300869,5.375010,5.303400,0
5110,0.512711,-0.093947,-0.947215,-0.812463,-1.011136,-1.008327,-0.936953,0.429860,-0.883867,1.160668,...,5.610130,5.334398,5.059620,5.088842,4.990484,5.314866,5.314547,5.045919,5.006436,0


In [26]:
# 데이터 섞기
data = data.sample(frac=1).reset_index(drop=True)
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_C4,spectral_entropy_T8,spectral_entropy_P7,spectral_entropy_P3,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,label
0,-0.251973,-0.344971,4.052259,-0.400547,-0.216186,-0.510230,0.408291,-0.009273,-0.380153,-0.105978,...,2.983482,3.125082,3.087301,3.063803,3.281389,3.297128,3.930503,3.484287,3.657525,1
1,-1.476762,-0.811918,1.244157,-1.406731,-0.024212,-1.207480,-0.099610,1.134932,-1.020751,-1.062254,...,3.837522,5.039702,5.110938,5.098228,5.151971,5.041034,5.125718,4.806836,4.815127,0
2,-0.458526,0.733854,0.313247,2.877850,-0.827374,2.157928,-0.343658,-0.065857,-0.655495,-0.890905,...,4.470006,4.783893,4.675387,4.288468,4.371660,4.446128,5.085434,4.350697,3.938208,1
3,-1.325012,-2.580050,-1.756758,-0.915368,0.186652,-0.460590,-0.783273,0.429390,0.403335,0.906941,...,2.680160,2.710314,2.827229,2.762234,2.748323,2.767089,2.876978,2.963769,2.949423,0
4,-1.343042,-1.549083,0.536665,-1.290034,-0.502404,-0.544905,1.727487,0.364965,0.353710,-0.482214,...,4.618227,4.116095,3.808181,3.883364,4.428648,4.306426,3.209085,3.457619,3.370598,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5107,-0.352343,-0.795385,0.084062,-0.508689,-0.577231,-0.507277,-0.204168,-0.594964,-1.130960,-0.743310,...,2.205138,2.252245,2.278385,2.242072,2.215098,2.224424,2.270710,2.282142,2.272563,2
5108,0.651278,-1.046622,2.115584,-1.310074,-0.448948,-0.660012,-1.548277,-0.863483,-1.216963,-0.178634,...,4.706900,4.763663,4.730680,4.698088,4.747524,4.813438,4.965626,4.825639,4.887476,1
5109,-1.099540,-1.508835,-1.426281,-1.099362,-0.900080,-1.224930,-0.886908,1.453205,-0.558681,0.225385,...,3.259754,3.349325,3.570600,3.521091,3.432351,3.416938,3.598937,3.692848,3.592018,0
5110,-1.562392,-0.264910,-0.044274,0.366741,-0.657590,0.105626,1.615998,-1.494533,-0.581559,-0.682188,...,5.004144,5.382321,5.537585,5.394393,4.664651,4.891490,5.304183,4.735151,4.749898,1


In [27]:
# 데이터 나누기
X = data.drop(['label'], axis=1)
y = data['label']
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [28]:
lasso = Lasso(alpha=0.01, max_iter=10000)
# SelectFromModel 학습
feature_selector = SelectFromModel(estimator=lasso)
feature_selector.fit(X_train, y_train)  # Lasso 모델 학습

SelectFromModel(estimator=Lasso(alpha=0.01, max_iter=10000))

## 1-1. L1 Regularization + RF (Grid)

In [29]:
# Random Forest 모델
rf_model = RandomForestClassifier(random_state=42)

# 파이프라인 구축
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('random_forest', rf_model)
])

# 하이퍼파라미터 그리드 정의
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1, 1],  # Lasso의 alpha 값
    'random_forest__n_estimators': [50, 100, 200],  # Random Forest의 트리 개수
    'random_forest__max_depth': [None, 10, 20, 30]  # Random Forest의 최대 깊이
}
# Weighted F1 스코어 사용
f1_scorer = make_scorer(f1_score, average='weighted')

grid_search_rf = GridSearchCV(pipeline, param_grid, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [30]:
# 모델 학습
grid_search_rf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(alpha=0.01,
                                                                        max_iter=10000))),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'feature_selection__estimator__alpha': [0.001, 0.01,
                                                                 0.1, 1],
                         'random_forest__max_depth': [None, 10, 20, 30],
                         'random_forest__n_estimators': [50, 100, 200]},
             scoring=make_scorer(f1_score, average=weighted), verbose=1)

In [31]:
# 최적 파라미터 및 성능 출력
print("Best Parameters:", grid_search_rf.best_params_)
print("Best CV Score:", grid_search_rf.best_score_)

# 최적 모델 로드
best_model_rf = grid_search_rf.best_estimator_

# 테스트 세트 평가
y_pred = best_model_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Test Accuracy: {accuracy:.2f}", f"Test F1: {f1:.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'random_forest__max_depth': None, 'random_forest__n_estimators': 100}
Best CV Score: 0.9303511541252716
Test Accuracy: 0.92 Test F1: 0.92


In [32]:
# 최적 모델 저장
joblib.dump(best_model_rf, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1rf_model_ver1.pkl")
print("Model saved to 'best_l1rf_model_ver1.pkl'")

Model saved to 'best_l1rf_model_ver1.pkl'


## 1-2. L1 Regularization + XGB (Grid)

In [33]:
# XGBoost 모델
xgb_model = XGBClassifier(
    objective='logloss',
    use_label_encoder=False,
    eval_metric='logloss'
)

# 파이프라인
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('xgboost', xgb_model)
])

# Grid Search 설정
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1],
    'xgboost__n_estimators': [50, 100, 200],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__learning_rate': [0.01, 0.1, 0.3]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

grid_search_xgb = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    verbose=1,
    n_jobs=-1
)

In [34]:
# 모델 학습
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(alpha=0.01,
                                                                        max_iter=10000))),
                                       ('xgboost',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
              

In [35]:
# 최적 파라미터 출력
print("Best Parameters:", grid_search_xgb.best_params_)

# 테스트 데이터 평가
y_pred = grid_search_xgb.best_estimator_.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Test F1-Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'xgboost__learning_rate': 0.3, 'xgboost__max_depth': 5, 'xgboost__n_estimators': 200}
Test Accuracy: 0.95
Test F1-Score: 0.95


In [36]:
# 최적 모델 저장
joblib.dump(grid_search_xgb.best_estimator_, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1xgb_model_ver1.pkl")
print("Model saved to 'best_l1xgb_model_ver1.pkl'")

Model saved to 'best_l1xgb_model_ver1.pkl'


# 나머지 절반 NC 데이터 테스트

## Version 1

In [39]:
new_data_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver1(half NC)/features_nc2.csv'
new_data = pd.read_csv(new_data_dir)
new_data_X = new_data.drop(['label'], axis=1)
new_data_y = new_data['label']

In [42]:
y_pred_rf_nc2 = grid_search_rf.best_estimator_.predict(new_data_X)
y_pred_xgb_nc2 = grid_search_xgb.best_estimator_.predict(new_data_X)

In [44]:
acc_rf_nc2 = accuracy_score(new_data_y, y_pred_rf_nc2)
f1_rf_nc2 = f1_score(new_data_y, y_pred_rf_nc2, average='weighted')

acc_xgb_nc2 = accuracy_score(new_data_y, y_pred_xgb_nc2)
f1_xgb_nc2 = f1_score(new_data_y, y_pred_xgb_nc2, average='weighted')

print(f"rf. Test Accuracy: {acc_rf_nc2:.2f}", f"Test F1: {f1_rf_nc2:.2f}")
print(f"xgb. Test Accuracy: {acc_xgb_nc2:.2f}", f"Test F1: {f1_xgb_nc2:.2f}")

rf. Test Accuracy: 0.30 Test F1: 0.46
xgb. Test Accuracy: 0.28 Test F1: 0.44
